1.生成p和q大质数

In [4]:
import gmpy2
import random

def generate_large_prime(bits=1024):
    return gmpy2.next_prime(random.getrandbits(bits))

p = generate_large_prime(1024)
q = generate_large_prime(1024)

print("p =", p)
print("q =", q)

p = 56632976249584754274316733999592386419720100331515095444735830885675276019534173263110027897235485886220284648971727820559115740536836367727681070478635408354957819995733965073278256948170199179439420726989848176430573486243579714708527981725580108927265085323969403983729070882797152218062977817198901291181
q = 19134250024328708356173639309818122561792574603563805259751909035633794930768279035205153447690526891722860816936027439024515715299557776491318690349927570977668832442009047404918325937124763879735944811762068539807693256270289673585013312625838458305923395852479466553094402897884939489580628793086492080391


2.根据p和q生成n、d、e，其中公钥是(n,e)，私钥是(n,d)。

In [12]:
#!python setup.py install #安装依赖
!python rsatool.py -p {p} -q {q} -o private.pem

Using (p, q) to calculate RSA paramaters

n =
89580fb28fb971248fbfbacfd9ea85fc52cd5ed68e875bbf30ad67e621651dceca9b312c2bce5aea
c0f34ea7b70edd5bcaf472255a16a4c8c7f2f600311b070ab20d0fb0cec3b1149082a0019c4955c8
f0c1fb6e8ec6459885fd0a85d615bed8ce27c180c855713e43704b1d06f1708030dd1fec7b6ffab2
c657c3a1ecd3f4a751c6f6903d4ec2fbee4d67da7b9141715681d08fb71ec74e7ff18818da3bb8e9
6676d5ae671a80ebf36d0606ac7c013fc66aa0a4f3c5acde0b80e61f8d024fe1aed114717f906017
ceeccc309bd40c6dd037e04e37c5279b7dbd4c8495f4725a062e00a9b692b9a6c8571ca79b22bf68
eba26e8afabd523758e7a22970021bb

e = 65537 (0x10001)

d =
6c624810fb86b618ca2fddb6851ba41c9923648220fc0494fe4ef9bd0c0185e258d4db9b25feb494
a92bd89855c7af3bef0fb4bc71398954c125a4a84618341866a61409cd41750eeb7389dd7b7e0bea
84c4302cf112da7d20cfad6c2ca400758b4aa8537ec49e972a1505d1a4570816c178dee67a0ac666
2a4bbc209e8a2d7442a5f4e6583ed2e80ac456ce3abaa8464472e0a1ed4d87eb77403e7111c4d1a2
a52cae0308e37d62502cafa5439cd40fc8b110bb216f8c80d63857d8a8036873e7a084f733ec9437
9d067

3.从私钥文件当中提取公钥。（私钥文件中存有公钥（就是公钥指数e和n））

In [13]:
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.serialization import load_pem_private_key

# 1. 加载私钥文件
with open("private.pem", "rb") as f:
    private_key = load_pem_private_key(f.read(), password=None)

# 2. 提取公钥
public_key = private_key.public_key()

# 3. 将公钥保存为 PEM 格式文件
pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

with open("public.pem", "wb") as f:
    f.write(pem)

print("公钥已成功提取并保存为 public.pem")

公钥已成功提取并保存为 public.pem


4.加载公钥，并对信息（message变量）进行加密。这里用的是OAEP填充，使用了哈希算法和随机数，因此每次密文不同。

In [14]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding

# -------------------------------
# 步骤1：从 PEM 文件中加载公钥
# -------------------------------

with open("public.pem", "rb") as f:
    public_key = serialization.load_pem_public_key(f.read())

# -------------------------------
# 步骤2：定义要加密的消息（必须是 bytes 类型）
# -------------------------------

message = "原神启动！".encode('utf-8')

# -------------------------------
# 步骤3：使用公钥进行 RSA 加密（OAEP 填充）
# -------------------------------

encrypted = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# -------------------------------
# 步骤4：输出加密结果（通常以十六进制或 Base64 编码传输）
# -------------------------------

print("加密后的数据（Hex 编码）:")
print(encrypted.hex())

# 保存加密结果到文件
with open("encrypted.bin", "wb") as f:
    f.write(encrypted)

print("加密完成，并已保存到 encrypted.bin")

加密后的数据（Hex 编码）:
10dc5091b5ec70467fd188db7789960f45dba9d8e81647693740e7f65f87f019ff41e425daa0a67262ceb9e342e488727789040c77a135fb4e3af692824eee64936069528aaa6108598c90a6a2292d592d4ef3f7da8cf9c7b9d303cb0684791639acabb27d089dfb9d6ffafb9227a56eeb69e23f9453c41f1aa935c4c1927047cbf05c533beffa2345a81c24927ac0a84158ac216fd50d5c687b48f7e69af3629cfa78e5682abfb2942a25bc17d73b1655ecdae54519bf96c9eb2c230fd4e40966bd5792b57ec7310065a2d52791d37bffb99e8106b0d32983f5c7af5a84ea8c380125795cbc53e6cca3f60fe7ef50213572a59bb69c45f9e894d2eaf210f85a
加密完成，并已保存到 encrypted.bin


5.读取密文，使用私钥解密。

In [19]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding

# -------------------------------
# 步骤1：从 PEM 文件中加载私钥
# -------------------------------

with open("private.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None  # 如果私钥有密码保护，请传入相应密码
    )

# -------------------------------
# 步骤2：读取加密的数据（假设是 Hex 编码）
# -------------------------------
with open("encrypted.bin", "rb") as f:
    encrypted = f.read()

# -------------------------------
# 步骤3：使用私钥进行 RSA 解密（OAEP 填充）
# -------------------------------

decrypted = private_key.decrypt(
    encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# -------------------------------
# 步骤4：输出解密后的消息
# -------------------------------

print("解密后的消息：")
print(decrypted.decode('utf-8'))

解密后的消息：
原神启动！


# 总结
1. 生成p、q。  
2. 根据p、q生成密钥对并保存在private.pem当中。  
3. 从私钥文件中读取公钥，并保存在public.pem当中。  
4. 使用public.pem对信息进行加密，当中使用OAEP填充方法，并保存在encrypted.bin当中。  
5. 读取private.pem的私钥，并使用私钥解密信息并打印。  
  
# 补充
1. 裸rsa存在隐患，即相同的明文会生成相同的密文，可以被多种手段破解。因此需要引入随机的因素，OAEP就是目前最主流最安全的填充方法。虽然这是一个可逆的过程，但是可以**防止确定性加密攻击**和**增强抗选择性密文攻击能力**。    
2. 全程使用utf-8编码，OAEP填充的参数mgf是MGF1(SHA-256)，主哈希是SHA-256。说是哈希算法保持一致比较常见。mgf是一个伪随机哈希函数，根据它的哈希算法和要求的长度参数生成指定长度的伪随机序列。  
3. 2048位、1024位，这里的位指的是二进制位，十进制下大概有309和617位。  

In [ ]:
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding

# -------------------------------
# 步骤1：从 PEM 文件中加载私钥
# -------------------------------

with open("ttt.pem", "rb") as f:
    private_key = serialization.load_pem_private_key(
        f.read(),
        password=None  # 如果私钥有密码保护，请传入相应密码
    )

# -------------------------------
# 步骤2：读取加密的数据（假设是 Hex 编码）
# -------------------------------
with open("tt.bin", "rb") as f:
    encrypted = f.read()

# -------------------------------
# 步骤3：使用私钥进行 RSA 解密（OAEP 填充）
# -------------------------------

decrypted = private_key.decrypt(
    encrypted,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# -------------------------------
# 步骤4：输出解密后的消息
# -------------------------------

print("解密后的消息：")
print(decrypted.decode('utf-8'))